In [ ]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import altair as alt
import pandas as pd
# from vega_datasets import data
from dash.dependencies import Input, Output

##########################
df = pd.read_csv('Sleep_Efficiency.csv')

## Wrangle Data 
# Handling na, strategy = replaced by mean
df['Alcohol consumption'].fillna(df['Alcohol consumption'].mean(),inplace=True)
df['Caffeine consumption'].fillna(df['Caffeine consumption'].mean(),inplace=True)
df['Awakenings'].fillna(round(df['Awakenings'].mean()),inplace=True)
df['Exercise frequency'].fillna(round(df['Exercise frequency'].mean()),inplace=True)

# df['Awakenings'].value_counts()
# df.isnull().sum()

# Bin ages into 11 groups, Create new df col binnedAge
age_bins = [0] + [5*i for i in range(3,13)] + [df.Age.max()]
age_bins_labels = ['0-15','15-20','21-25','26-30','31-35','36-40','41-45','46-50','51-55','56-60','60+']
df['binnedAge'] = pd.cut(df['Age'],bins=age_bins,labels=age_bins_labels)

################################
## Define Charts


def awakening_bar():
    chart = alt.Chart(df).mark_bar().encode(
        x = 'count()',
        y = alt.Y('Awakenings:O'),
        color = alt.Color('Gender')
    )
    return chart.to_html()

################################
## Define Layouts
# Tentative, appropriate css to be set
app=dash.Dash(__name__,external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])

app.layout = html.Div([
    # Left Div : Input Controls
    html.Div([html.H1('Sleep Efficiency Dash2', style={'backgroundColor':'white'}),
             html.P("Dataset Source:"),
             html.A("Sleep Efficiency Dataset by EQUILIBRIUMM", href="https://www.kaggle.com/datasets/equilibriumm/sleep-efficiency"),
             html.Br(),
             html.Br(),
             html.Br(),
             html.Br(),
              
              
             html.Div(id='check-input-value',children='check value'),
             html.P('Smoke Status', style={"font-weight": "bold"}),
             dcc.Checklist(
                 id='smoke-status-input',
                 options=['Yes','No'],
                 value=['Yes','No'],
                 inline = True),
              
              html.Br(),
              
             html.P('Alcohol Consumption', style={"font-weight": "bold"}),
             dcc.RangeSlider(
                 id='alcohol-consumption-input',
                 min=0,max=5,step=1,value=[0,5],
                 marks={
                     0: {'label':'0mg', 'style': {'color': '#77b0b1'}},
                     1: {'label':'1mg'},
                     2: {'label':'2mg'},
                     3: {'label':'3mg'},
                     4: {'label':'4mg'},
                     5: {'label':'5mg', 'style': {'color': '#f50'}}
                 }
                 ),
              
              html.Br(),
                            
             html.P('Caffeine Consumption', style={"font-weight": "bold"}),
             dcc.RangeSlider(
                 id='caffeine-consumption-input',
                 min=0,max=200,step=25,value=[0,200],
                 marks={
                     0: {'label':'0oz', 'style': {'color': '#77b0b1'}},
                     25: {'label':'25oz'},
                     50: {'label':'50oz'},
                     75: {'label':'75oz'},
                     100: {'label':'100oz'},
                     125: {'label':'125oz'},
                     150: {'label':'150oz'},
                     175: {'label':'175oz'},
                     200: {'label':'200mg', 'style': {'color': '#f50'}}
                 }),
              
              html.Br(),
              
             html.P('Execise Frequency (each week)', style={"font-weight": "bold"}),
             dcc.RangeSlider(
                 id='exercise-frequency-input',
                 min=0,max=5,step=1,value=[0,5],
                 marks={
                     0: {'label':'0', 'style': {'color': '#77b0b1'}},
                     1: {'label':'1 time'},
                     2: {'label':'2 time'},
                     3: {'label':'3 time'},
                     4: {'label':'4 time'},
                     5: {'label':'5 time', 'style': {'color': '#f50'}}
                 })
             ],
            style={'width': '25%', 'display': 'inline-block'}),
    
    # Right Main Div 
    html.Div([
        # Right Top
        html.Div([
            
            # Right Top Left
            html.Div([
                'Right Top Left Section',
                html.Br(),
                html.Iframe(
                    id = 'age_sleepeff',
#                    srcDoc=age_sleepeff_line(),
                    style={'height':'500px','width':'100%'}
                )],style={'width': '50%', 'display': 'inline-block'}
            ),
            
            # Right Top Right
            html.Div([
                'Right Top Right Section',
                html.Br(),
                html.Iframe(
                    id = 'age_sleepdur',
#                    srcDoc=age_sleepdur_bar(),
                    style={'height':'500px','width':'100%'} #,'float':'right', 'display': 'inline-block'}
                )], style={'width': '50%','display': 'inline-block'}
            )]         
        ),
        
        # Right Bottom
        html.Div([
            'graph bottom',
            html.Iframe(
                id = 'awakening',
                srcDoc=awakening_bar(),
                style={'width': '100%','height':'500px', 'display': 'inline-block'}
            )
        ])],
        style={'width': '75%', 'float':'right','display': 'inline-block'}
    )]
)


# Ref: https://github.com/Coding-with-Adam/Dash-by-Plotly/blob/master/Dash%20Components/Checklist/checkbox.py

@app.callback(
    Output('age_sleepeff','srcDoc'),
    Output('age_sleepdur','srcDoc'),
    Output('awakening','srcDoc'),
#    Output('check-input-value','children'),
    Input('smoke-status-input','value'),
    Input('alcohol-consumption-input','value'),
    Input('caffeine-consumption-input','value'),
    Input('exercise-frequency-input','value'))


def update_output(smoke_status,alcohol_consumption,caffeine_consumption,exercise_frequency):
    df_local = df[df['Smoking status'].isin(smoke_status)].copy()
    df_local = df_local[df_local['Alcohol consumption'].between(alcohol_consumption[0],alcohol_consumption[1])]
    df_local = df_local[df_local['Caffeine consumption'].between(caffeine_consumption[0],caffeine_consumption[1])]
    df_local = df_local[df_local['Exercise frequency'].between(exercise_frequency[0],exercise_frequency[1])]
    
    domain = ['Male', 'Female']
    range_ = ['#3493bf', '#c74448']
    
    age_sleepeff_point = alt.Chart(df_local).mark_point().encode(
        x = alt.X('binnedAge', title = "Age Groups", axis=alt.Axis(titleFontSize=20)),
        y = alt.Y('mean(Sleep efficiency)',title = "Sleep Efficiency(Mean)", axis=alt.Axis(titleFontSize=20), scale=alt.Scale(zero=False)),
        color = alt.Color('Gender', scale=alt.Scale(domain=domain, range=range_))
    ).properties(
    height=400, width=390
    )
    
    age_sleepeff_line = age_sleepeff_point.mark_line().properties(
    height=400, width=390
    )
    age_sleepeff_line2 = age_sleepeff_point + age_sleepeff_line
    
    age_sleepeff_line2.configure(background='#fafdff').properties(
    height=400, width=390
    ).configure(
    background='#EAEDED'
    ).configure_axis(
    labelFontSize=14,
    titleFontSize=12
    ).configure_title(
        fontSize=20
    ).interactive()
    

    
    age_sleepdur_bar = alt.Chart(df_local).mark_bar().encode(
        x = alt.X('Gender',title=None, axis=alt.Axis(grid=False, ticks=False,labels=False)),
        y = alt.Y('mean(Sleep duration)',scale=alt.Scale(zero=False),axis=alt.Axis(grid=False)),
        color = alt.Color('Gender',legend=alt.Legend(orient='top'),scale=alt.Scale(domain=domain, range=range_)),
        column='binnedAge'
    ).configure_view(
        stroke='transparent'
    ).configure(
    background='#EAEDED'
    ).configure_axis(
    labelFontSize=10,
    titleFontSize=17
    ).configure_title(
        fontSize=20
    ).interactive().properties(
    height=350, width=22
    )
    
    awakening_bar = alt.Chart(df_local).mark_bar().encode(
        x = 'count()',
        y = alt.Y('Awakenings:O'),
        color = alt.Color('Gender',legend=alt.Legend(orient='top'),scale=alt.Scale(domain=domain, range=range_))
    ).properties(
    height=250, width=900
    ).configure_axis(
    labelFontSize=14,
    titleFontSize=15
    ).configure_title(
        fontSize=20
    ).interactive()
    
    
    
#    return age_sleepeff_line2.to_html(), age_sleepdur_bar.to_html(), awakening_bar.to_html(),'From ' + str(alcohol_consumption[0]) + ' to ' + str(alcohol_consumption[1])
    return age_sleepeff_line2.to_html(), age_sleepdur_bar.to_html(), awakening_bar.to_html()

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)  

C:\Users\cmton\AppData\Local\Temp\ipykernel_23768\3166945178.py:2: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\cmton\AppData\Local\Temp\ipykernel_23768\3166945178.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
